In [1]:
import re

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import *

In [2]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [3]:
def cleanTweet(tweet: str) -> str:
    tweet = re.sub(r'http\S+', '', str(tweet))
    tweet = re.sub(r'bit.ly/\S+', '', str(tweet))
    tweet = tweet.strip('[link]')

    # remove users
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', str(tweet))
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', str(tweet))

    # remove puntuation
    my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@â'
    tweet = re.sub('[' + my_punctuation + ']+', ' ', str(tweet))

    # remove number
    tweet = re.sub('([0-9]+)', '', str(tweet))

    # remove hashtag
    tweet = re.sub('(#[A-Za-z]+[A-Za-z0-9-_]+)', '', str(tweet))

    return tweet


In [28]:
spark = SparkSession\
        .builder\
        .appName("TwitterSentimentAnalysis")\
        .master("local[*]")\
        .getOrCreate()

In [29]:
spark.sparkContext.setLogLevel('ERROR')

In [30]:
df = spark.readStream\
        .format("socket")\
        .option("host", "127.0.0.1")\
        .option("port", 3333)\
        .load()

In [31]:
df.printSchema()

root
 |-- value: string (nullable = true)



In [32]:
tweet_schema = StructType().add("ID", "string").add("text", "string").add("created_at", "string")

In [33]:
values = df.select(from_json(df.value.cast("string"), tweet_schema).alias("tweet"))

In [34]:
values.printSchema()

root
 |-- tweet: struct (nullable = true)
 |    |-- ID: string (nullable = true)
 |    |-- text: string (nullable = true)
 |    |-- created_at: string (nullable = true)



In [35]:
df1 = values.select("tweet.*")
clean_tweets = F.udf(cleanTweet, StringType())
raw_tweets = df1.withColumn('processed_text', clean_tweets(col("text")))

In [36]:
raw_tweets.printSchema()

root
 |-- ID: string (nullable = true)
 |-- text: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- processed_text: string (nullable = true)



In [ ]:
    writeTweet = raw_tweets.writeStream. \
    outputMode("update"). \
    format("console"). \
    trigger(processingTime='2 seconds'). \
    start()
    
    writeTweet.awaitTermination()

In [39]:
spark.stop()